In [1]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("codellama/CodeLlama-7b-hf", load_in_4bit=True, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")

In [3]:
PROMPT = '''
def remove_non_ascii(s: str) -> str:
  """ <FILL_ME>
  return result
'''

input_ids = tokenizer(PROMPT, return_tensors="pt")["input_ids"]
input_ids.to('cuda')
generated_ids = model.generate(input_ids, max_new_tokens=128)

filling = tokenizer.batch_decode(generated_ids[:, input_ids.shape[1]:], skip_special_tokens = True)[0]
print(PROMPT.replace("<FILL_ME>", filling))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/home/kiddos/.pyenv/versions/3.11.4/lib/python3.11/site-packages/transformers/generation/utils.py:1539: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/home/kiddos/.pyenv/versions/3.11.4/lib/python3.11/site-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or


def remove_non_ascii(s: str) -> str:
  """ Remove non-ASCII characters from a string.

  Args:
    s: The string to remove non-ASCII characters from.

  Returns:
    The string with non-ASCII characters removed.
  """
  result = ''
  for c in s:
    if ord(c) < 128:
      result += c
  return result



In [4]:
fill = '''<FILL_ME>'''
input_ids = tokenizer(fill, return_tensors="pt")["input_ids"]
print(input_ids)

tensor([[1]])


In [5]:
fill = '''"""<FILL_ME>'''
input_ids = tokenizer(fill, return_tensors="pt")["input_ids"]
print(input_ids)

tensor([[   1, 9995]])


In [6]:
fill = '''"""'''
input_ids = tokenizer(fill, return_tensors="pt")["input_ids"]
print(input_ids)

tensor([[   1, 9995]])


In [7]:
x = '''
def remove_non_ascii(s: str) -> str:
  """ <FILL_ME>
'''
input_ids = tokenizer(x, return_tensors="pt")["input_ids"]
print(input_ids)

tensor([[    1, 32007, 29871,    13,  1753,  3349, 29918,  5464, 29918,   294,
         18869, 29898, 29879, 29901,   851, 29897,  1599,   851, 29901,    13,
         29871,  9995, 29871, 32008,    13, 32009]])


In [8]:
x = '''
def remove_non_ascii(s: str) -> str:
'''
input_ids = tokenizer(x, return_tensors="pt")["input_ids"]
print(input_ids)

tensor([[    1, 29871,    13,  1753,  3349, 29918,  5464, 29918,   294, 18869,
         29898, 29879, 29901,   851, 29897,  1599,   851, 29901,    13]])


In [9]:
from transformers import pipeline
from transformers.pipelines.text_generation import ReturnType
import torch

generator = pipeline(
  "text-generation",
  model=model,
  tokenizer=tokenizer,
)

generator(
  PROMPT,
  max_new_tokens=128,
  return_type=ReturnType.NEW_TEXT
)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': 'Remove non-ASCII characters from a string. """\n  result = \'\'\n  for c in s:\n    if ord(c) < 128:\n      result += c'}]